In [5]:
import pandas as pd
import numpy as np

In [25]:
# Load DF
activity = pd.read_csv("Project/activites_cleaned.csv")
app_usage = pd.read_csv("Project/apps_usage_cleaned.csv")
calls = pd.read_csv("Project/calls.csv")
location = pd.read_csv("Project/location_cleaned.csv")
phone_calls = pd.read_csv("Project/phone_calls.csv")
app_info = pd.read_csv("app_crawler/crawled_app_info.csv")

In [26]:
# Sample data (replace this with your actual data)
data = pd.merge(app_usage, app_info, on = "package_name")
df = pd.DataFrame(data)

# Convert time_of_event to datetime and extract day of the week
df['applicationCategory'] = df['applicationCategory'].str.replace(r'GAME_\w+', 'GAME', regex=True)
df['time_of_event'] = pd.to_datetime(df['time'])
df['day_of_week'] = df['time_of_event'].dt.dayofweek
df['is_weekend'] = np.where(df['day_of_week'].isin([5, 6]), 1, 0)
df['hour_of_day'] = df['time_of_event'].dt.hour

df.head()

,time,package_name,last_time_used,foreground_time_ms,user_id,@context,@type,name,url,description,...,applicationCategory,image,contentRating,author,aggregateRating,offers,time_of_event,day_of_week,is_weekend,hour_of_day
0,2018-04-24 06:07:19.846,com.google.android.youtube,2018-04-22 23:15:39.878,857910,user_0,https://schema.org,SoftwareApplication,YouTube,https://play.google.com/store/apps/details/You...,Enjoy your favorite videos and channels with t...,...,VIDEO_PLAYERS,https://play-lh.googleusercontent.com/6am0i3wa...,Teen,"{'@type': 'Person', 'name': 'Google LLC', 'url...","{'@type': 'AggregateRating', 'ratingValue': '3...","[{'@type': 'Offer', 'price': '0', 'priceCurren...",2018-04-24 06:07:19.846,1,0,6
1,2018-04-24 06:07:19.846,com.whatsapp,2018-04-23 10:49:19.108,4525213,user_0,https://schema.org,SoftwareApplication,WhatsApp Messenger,https://play.google.com/store/apps/details/Wha...,Simple. Reliable. Private.,...,COMMUNICATION,https://play-lh.googleusercontent.com/bYtqbOcT...,Everyone,"{'@type': 'Person', 'name': 'WhatsApp LLC', 'u...","{'@type': 'AggregateRating', 'ratingValue': '4...","[{'@type': 'Offer', 'price': '0', 'priceCurren...",2018-04-24 06:07:19.846,1,0,6
2,2018-04-24 06:07:19.846,com.robtopx.geometryjump,2018-04-22 22:00:02.325,6365377,user_0,https://schema.org,SoftwareApplication,Geometry Dash,https://play.google.com/store/apps/details/Geo...,Jump and fly your way through danger in this r...,...,GAME,https://play-lh.googleusercontent.com/ixHXzBWP...,Everyone 10+,"{'@type': 'Person', 'name': 'RobTop Games', 'u...","{'@type': 'AggregateRating', 'ratingValue': '4...","[{'@type': 'Offer', 'price': '2.99', 'priceCur...",2018-04-24 06:07:19.846,1,0,6
3,2018-04-24 06:07:19.846,com.instagram.android,2018-04-23 09:16:34.899,1584815,user_0,https://schema.org,SoftwareApplication,Instagram,https://play.google.com/store/apps/details/Ins...,"Create & share photos, stories, & reels with f...",...,SOCIAL,https://play-lh.googleusercontent.com/VRMWkE5p...,Teen,"{'@type': 'Person', 'name': 'Instagram', 'url'...","{'@type': 'AggregateRating', 'ratingValue': '3...","[{'@type': 'Offer', 'price': '0', 'priceCurren...",2018-04-24 06:07:19.846,1,0,6
4,2018-04-24 06:07:19.846,it.vodafone.my190,2018-04-23 05:30:22.982,16942,user_0,https://schema.org,SoftwareApplication,My Vodafone Italia,https://play.google.com/store/apps/details/My_...,With My Vodafone app you have everything at yo...,...,TOOLS,https://play-lh.googleusercontent.com/C1zufX9D...,Everyone,"{'@type': 'Person', 'name': 'Vodafone Italia S...","{'@type': 'AggregateRating', 'ratingValue': '2...","[{'@type': 'Offer', 'price': '0', 'priceCurren...",2018-04-24 06:07:19.846,1,0,6


In [27]:
# Convert 'time' columns to datetime
df['time'] = pd.to_datetime(df['time'])
location['time'] = pd.to_datetime(location['time'])

# Round 'time' column to nearest hour
df['rounded_time'] = df['time'].dt.round('H')
location['rounded_time'] = location['time'].dt.round('H')

df = df.groupby(['rounded_time', 'package_name', 'user_id', 'applicationCategory'])['foreground_time_ms'].sum().reset_index()
location = location[['rounded_time', 'label', 'user_id']].drop_duplicates()

# Merge on 'rounded_time' and 'user_id'
merged_df = pd.merge(df, location, on=['rounded_time', 'user_id'])

merged_df.head()

C:\Users\wongh\AppData\Local\Temp\ipykernel_25976\816268356.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['rounded_time'] = df['time'].dt.round('H')
C:\Users\wongh\AppData\Local\Temp\ipykernel_25976\816268356.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  location['rounded_time'] = location['time'].dt.round('H')


,rounded_time,package_name,user_id,applicationCategory,foreground_time_ms,label
0,2018-03-27 09:00:00,co.squidapp.squid,user_10,NEWS_AND_MAGAZINES,2540265,free_time
1,2018-03-27 09:00:00,co.squidapp.squid,user_10,NEWS_AND_MAGAZINES,2540265,external_school
2,2018-03-27 09:00:00,com.amazon.mShop.android.shopping,user_10,SHOPPING,21228725,free_time
3,2018-03-27 09:00:00,com.amazon.mShop.android.shopping,user_10,SHOPPING,21228725,external_school
4,2018-03-27 09:00:00,com.android.chrome,user_12,COMMUNICATION,5178908,external_school


In [11]:
df.to_csv("app_usage_app_info.csv")

In [28]:
merged_df.to_csv("app_usage_location.csv")

# Weekend vs Weekday

In [38]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Sample data (replace this with your actual data)
data = pd.merge(app_usage, app_info, on = "package_name")
df = pd.DataFrame(data)

# Convert time_of_event to datetime and extract day of the week
df['time_of_event'] = pd.to_datetime(df['time'])
df['day_of_week'] = df['time_of_event'].dt.dayofweek
df['is_weekend'] = np.where(df['day_of_week'].isin([5, 6]), 1, 0)

# Fill missing foreground_time with 0
df['foreground_time_ms'].fillna(0, inplace=True)

df = df.groupby(['user_id', 'day_of_week', 'is_weekend', 'applicationCategory'])['foreground_time_ms'].sum().reset_index()

# Perform linear regression
X = pd.get_dummies(df['applicationCategory'])
X['is_weekend'] = df['is_weekend']
y = df['foreground_time_ms']

# Convert data to numeric types and fill NaN values
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
y = pd.to_numeric(y, errors='coerce').fillna(0)

X = sm.add_constant(X)  # Add constant term
model = sm.OLS(y.to_numpy(), X.to_numpy(dtype = float)).fit()
predictions = model.predict(X)

# Print regression results
print(model.summary(xname = X.columns.tolist()))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.294
Method:                 Least Squares   F-statistic:                     25.63
Date:                Mon, 28 Jul 2025   Prob (F-statistic):          1.72e-157
Time:                        15:46:24   Log-Likelihood:                -49405.
No. Observations:                2430   AIC:                         9.889e+04
Df Residuals:                    2388   BIC:                         9.914e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                3.939e+07   7

C:\Users\wongh\AppData\Local\Temp\ipykernel_19232\264343283.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['foreground_time_ms'].fillna(0, inplace=True)


In [68]:
pvalues[max_pvalue]

nan

In [72]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Sample data (replace this with your actual data)
data = pd.merge(app_usage, app_info, on="package_name")
df = pd.DataFrame(data)

# Convert time_of_event to datetime and extract day of the week
df['time_of_event'] = pd.to_datetime(df['time'])
df['day_of_week'] = df['time_of_event'].dt.dayofweek
df['is_weekend'] = np.where(df['day_of_week'].isin([5, 6]), 1, 0)

# Fill missing foreground_time with 0
df['foreground_time_ms'].fillna(0, inplace=True)

df = df.groupby(['user_id', 'day_of_week', 'is_weekend', 'applicationCategory'])['foreground_time_ms'].sum().reset_index()

# Perform linear regression
X = pd.get_dummies(df['applicationCategory'])
X['is_weekend'] = df['is_weekend']
y = df['foreground_time_ms']

# Convert data to numeric types and fill NaN values
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)
y = pd.to_numeric(y, errors='coerce').fillna(0)

X = sm.add_constant(X)  # Add constant term

while True:
    model = sm.OLS(y.to_numpy(), X.to_numpy(dtype=float)).fit()
    pvalues = model.pvalues
    max_pvalue = pvalues.argmax()
    
    if pvalues[max_pvalue] > 0.05 or pd.isna(pvalues[max_pvalue]):  # Adjust the significance level as needed
        X = X.drop(X.columns[max_pvalue], axis=1)
    else:
        break

# Fit the final model
X['is_weekend'] = df['is_weekend']
model = sm.OLS(y.to_numpy(), X.to_numpy(dtype=float)).fit()
predictions = model.predict(X)

# Print regression results
print(model.summary(xname=X.columns.tolist()))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     173.2
Date:                Mon, 28 Jul 2025   Prob (F-statistic):          1.07e-183
Time:                        16:59:37   Log-Likelihood:                -49414.
No. Observations:                2430   AIC:                         9.884e+04
Df Residuals:                    2423   BIC:                         9.888e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            1.871e+07   4.42e+06     

C:\Users\wongh\AppData\Local\Temp\ipykernel_19232\4111703508.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['foreground_time_ms'].fillna(0, inplace=True)


In [74]:
columns = X.columns
for i, c1_name in enumerate(columns):
    for j, c2_name in enumerate(columns):
        if j <= i or i == 0: continue
        interaction_col = X[c1_name] * X[c2_name]
        X[f'{columns[i]} X {columns[j]}'] = interaction_col

model = sm.OLS(y.to_numpy(), X.to_numpy(dtype=float)).fit()
predictions = model.predict(X)

# Print regression results
print(model.summary(xname=X.columns.tolist()))

while True:
    model = sm.OLS(y.to_numpy(), X.to_numpy(dtype=float)).fit()
    pvalues = model.pvalues
    max_pvalue = pvalues.argmax()
    
    if pvalues[max_pvalue] > 0.05 or pd.isna(pvalues[max_pvalue]):  # Adjust the significance level as needed
        X = X.drop(X.columns[max_pvalue], axis=1)
    else:
        break

model = sm.OLS(y.to_numpy(), X.to_numpy(dtype=float)).fit()
predictions = model.predict(X)

# Print regression results
print(model.summary(xname=X.columns.tolist()))

C:\Users\wongh\AppData\Local\Temp\ipykernel_19232\3243305785.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f'{columns[i]} X {columns[j]}'] = interaction_col
C:\Users\wongh\AppData\Local\Temp\ipykernel_19232\3243305785.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X[f'{columns[i]} X {columns[j]}'] = interaction_col
C:\Users\wongh\AppData\Local\Temp\ipykernel_19232\3243305785.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.301
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     94.54
Date:                Mon, 28 Jul 2025   Prob (F-statistic):          9.60e-179
Time:                        17:00:48   Log-Likelihood:                -49413.
No. Observations:                2430   AIC:                         9.885e+04
Df Residuals:                    2418   BIC:                         9.892e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                                                        coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

In [54]:
# Add variable interactions to the final model
for i in range(X.shape[1]):
    for j in range(i + 1, X.shape[1]):
        interaction_col = X.iloc[:, i] * X.iloc[:, j]
        X = pd.concat([X, interaction_col], axis=1)

model = sm.OLS(y, X).fit()
predictions = model.predict(X)

# Print regression results
print(model.summary(xname=X.columns.tolist()))

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [44]:
model.pvalues[1:].argmax()

11

In [49]:
X.

,const,ART_AND_DESIGN,AUTO_AND_VEHICLES,BOOKS_AND_REFERENCE,BUSINESS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,FINANCE,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,is_weekend
0,1.0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
1,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
2,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
3,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
4,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,1.0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2426,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2427,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
2428,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,1
